In [ ]:
// run this cell to prevent Jupyter from displaying the null output cell
com.twosigma.beakerx.kernel.Kernel.showNullExecutionResult = false;

<a id='notebook_id'></a>
# Floating-point errors

The fact that there are a finite number of floating-point values but an infinite number of real values implies that
the result of a floating-point computation will almost always contain some error from the true result. This notebook
describes some basic ideas in measuring floating-point errors, why errors occur when performing basic arithmetic,
and how to reduce or avoid such errors.

Throughout this notebook we use the three digit base-10 floating-point format described in the
[Representing real numbers on a computer](./floating_point_representation.ipynb#notebook_id) notebook.

This notebook is [based on the article](https://docs.oracle.com/cd/E19957-01/800-7895/800-7895.pdf)
*What Every Computer Scientist Should Know About Floating-Point Arithmetic* or WECSSKAFPA for short.


## Rounding error

When performing arithmetic with floating-point values, most calculations will produce quantities that cannot be exactly
represented (for example, see Exercise 13 of the [Representing real numbers on a computer](./floating_point_representation.ipynb#notebook_id) notebook). For example, the value $0.1$ cannot be represented exactly using a Java `double`; therefore, it is not too surprising that the following
program prints `false`:

In [3]:
double sum = 0.1 + 0.1 + 0.1;
System.out.println(sum == 0.3);

false


null

However, the following example prints `true`:

In [4]:
double sum = 0.1 + 0.1;
System.out.println(sum == 0.2);

true


null

as does the next example:

In [5]:
double sum = 0.1 + 0.1 + 0.1 + 0.1 + 0.1;
System.out.println(sum == 0.5);

true


null

In the first example, the rounding errors that accumulate in the sum produce a value that is not quite equal to the floating-point
value closest to 0.3 (which also cannot be represented exactly). In the second example, the rounding errors that accumulate in the sum
produce a value that is equal to the floating-point value closest to 0.2 (which also cannot be represented exactly).
In the third example, the rounding errors that accumulate in the sum
produce a value that is equal to the floating-point value closest to 0.5 (which can be represented exactly).

To be clear, the computed sums are very close to their expected values, but the reader should recognize that it is difficult
to predict the exact result of most floating-point calculations.

## Measuring error

The magnitude of rounding error or error in general is usually measured using *ulps* or *relative error*.

### Ulps

Ulps is shorthand for *units in the last place*. When using ulps, we need to consider the format of the floating-point
representation that we are using. For this notebook, assume that we have a base-10 floating-point format with $p=3$ significant digits (similar to
the format used in the previous notebook) and normalized values.

Suppose that we perform some calculation that produces the floating-point result $x$ and we know that the correct or true
result is the floating-point value $\hat{x}$. If we write $\hat{x}$ as a significand $s$

$$s = \pm\boxed{d_1}\boxed{d_2}\boxed{d_3}$$

and an exponent $e$ then 1 ulp is equal to $1 \times 10^e$. In other words, 1 ulp is the spacing between two consecutive floating-point
values for some exponent $e$. For example, $\hat{x}=1.29$ has the significand:

$$s = \boxed{1}\boxed{2}\boxed{9}$$

and exponent $e=-2$. The closest floating-point value to $\hat{x}$ is $1.30$ which has the significand:

$$s = \boxed{1}\boxed{3}\boxed{0}$$

and exponent $e=-2$. The difference between the two values is 1 unit in the last place of the significand which has a magnitude of $1 \times 10^{-2}$.

The error in ulps between a computed value $x$ and its true value $\hat{x}$ is the absolute value of the difference between
$\hat{x}$ and $x$:

$$\text{err} = \left| \hat{x} - x \right|$$

expressed as a multiple of $10^e$ where $e$ is the exponent of the floating-point representation of $\hat{x}$. When computing
the difference, assume that $\hat{x}$ and $x$ are real values so that the error is also a real value.

For example, suppose $\hat{x}=1.29$ and $x=1.25$; then:

$$\begin{split}
\text{err} & = \left| \hat{x} - x \right| \\
           & = \left| 1.29 - 1.25 \right| \\
           & = 0.04 \\
           & = 4 \times 10^{-2} \quad \text{difference expressed using the exponent of} \ \hat{x} \\
           & = 4\ \text{ulps}
\end{split}$$

In this example, we say that the error is equal to 4 ulps (units in the last place). Notice that the difference is expressed
using the exponent of the floating-point representation of $\hat{x} = 129 \times 10^{-2}$.

Notice that the location of the decimal point does not affect the number of ulps of error. For example, if $x = 125$ and $\hat{x} = 129$ then
the error is also 4 ulps (even though the absolute error is 100 times greater than in the previous example).

It is important to remember that we always use the exponent of the true value $\hat{x}$ to compute an error in ulps. 
For example, suppose $\hat{x}=12.1$ and $x=0.5$; then:

$$\begin{split}
\text{err} & = \left| \hat{x} - x \right| \\
           & = \left| 12.1 - 0.5 \right| \\
           & = 11.6 \\
           & = 116 \times 10^{-1} \quad \text{difference expressed using the exponent of} \ \hat{x} \\
           & = 116\ \text{ulps}
\end{split}$$

Here, we used the fact that $\hat{x}$ has the floating-point representation $121 \times 10^{-1}$.

If we know the $\hat{x}$ as a real value, then we can still measure error in ulps. Simply use the exponent
needed to express $\hat{x}$ using $p$ significant digits to the left of the decimal place.
For example, if the floating-point format has $3$ digits in the significand and $\hat{x} = 59287.5603$ then:

$$\begin{split}
\hat{x} & = 59287.5603 \\
        & = 592.875603 \times 10^{2} \quad \text{3 significant digits to the left of the decimal place}
\end{split}$$

and 1 ulp is equal to $1 \times 10^{2}$. If $x = 58900$ is the floating-point approximation of $\hat{x}$ then the error in ulps is

$$\begin{split}
\text{err} & = \left| \hat{x} - x \right| \\
           & = \left| 59287.5603 - 58900 \right| \\
           & = 387.5603 \\
           & = 3.875603 \times 10^{2} \\
           & = 3.875603\ \text{ulps}
\end{split}$$

If $\hat{x} = 0.035652$ then:

$$\begin{split}
\hat{x} & = 0.035652 \\
        & = 356.52 \times 10^{-4}
\end{split}$$

then 1 ulp is equal to $1 \times 10^{-4}$. If $x = 343 \times 10^{-4}$ is the floating-point approximation of $\hat{x}$ then the error in ulps is

$$\begin{split}
\text{err} & = \left| \hat{x} - x \right| \\
           & = \left| 356.52 \times 10^{-4} - 343 \times 10^{-4} \right| \\
           & = 13.53 \times 10^{-4} \\
           & = 13.53\ \text{ulps}
\end{split}$$


**Exercise 1** For each pair of exact values $\hat{x}$ and floating-point values $x$ compute the error in ulps.

| exact value $\hat{x}$ | floating-point value $x$ |
| :--- | :--- |
| 0.129 | 0.125 |
| 3.1416 | 3.1 |
| 0.9 | 5.78|
| 9.9999 | 10.0 |

**Exercise 2** 1 ulp is the distance between two consecutive floating-point values. If a true value $\hat{x}$ is exactly half way between
two consecutive floating-point values, what is the smallest possible rounding error in ulps?

**Exercise 3** If a true value $\hat{x}$ is *not* exactly half way between
two consecutive floating-point values, what can you say about the magnitude of the smallest possible rounding error in ulps?

### Relative error

*Relative error* is defined as the difference between the true value and the floating-point value divided by true value; i.e.

$$\text{relative error} = \left| \frac{\hat{x} - x}{\hat{x}} \right|$$

For example, when $x=1.2345$ and $\hat{x} = 1.2349$ the relative error is $0.0004 / 1.2349 \approx 0.0003239$. Note that relative
error is computed using real arithmetic instead of floating-point arithmetic; therefore, there is no need to consider the format
of the floating-point type being used.

As with ulps, the location of the decimal point does not affect the magnitude of the relative error. For example, whe
when $x=1234.5$ and $\hat{x} = 1234.9$ the relative error is $0.4 / 1234.9 \approx 0.0003239$.

Because of the division by $\hat{x}$, relative error can be very high when $\hat{x}$ is small even if the absolute error $\left| \hat{x} - x \right|$
is small.

**Exercise 4** Compute the relative errors for the values in Exercise 1.

**Exercise 5** What does a relative error equal to 1 mean? What does a relative error equal to 2 mean? What does a relative error equal to 3 mean? 

## Adding/subtracting two values

Adding or subtracting two floating-point values is fairly straightforward when the two operands have the same exponent. In many cases, adding or
subtracting the significands and copying the exponent yields the correct answer. For example, if $x=4.76$ and $y=3.23$ then $x+y$ is equal to:

$$\begin{split}
x & : s = \boxed{4}\boxed{7}\boxed{6}\quad e = -2 \\
y & : s = \boxed{3}\boxed{2}\boxed{3}\quad e = -2 \\
x + y & : s = \boxed{7}\boxed{9}\boxed{9}\quad e = -2 \\
\end{split}$$

Instead of drawing the significand, we can simply write the sum as:

$$\begin{split}
x & = 476 \times 10^{-2} \\
y & = 323 \times 10^{-2} \\
x + y &= 799 \times 10^{-2}
\end{split}$$

Subtraction when both operands have the same exponent works similarly to addition.

If a sum produces a fourth digit at the front of the significand, then for our purposes, the sum is computed using four digits and the final result is
rounded to fit into a three digit significand. For example, if $x=92.8$ and $y=37.8$ then $x+y$ is equal to:

$$\begin{split}
x & : s = \quad \boxed{9}\boxed{2}\boxed{8}\quad e = -1 \\
y & : s = \quad \boxed{3}\boxed{7}\boxed{8}\quad e = -1 \\
x + y & : s = \boxed{1}\boxed{3}\boxed{0}\boxed{6}\quad e = -1 \quad \text{intermediate result} \\
      & : s = \quad \boxed{1}\boxed{3}\boxed{1}\quad e = 0 \quad \text{final result (notice the adjusted exponent!)}
\end{split}$$

or:

$$\begin{split}
x & = 928 \times 10^{-1} \\
y & = 378 \times 10^{-1} \\
x + y & = 1306 \times 10^{-1} \quad \text{intermediate result} \\
      & = 131 \times 10^{0} \quad \text{final result}
\end{split}$$

If the exponents of the two operands are different then the operand with the smaller exponent is converted to a value having the same
exponent as the operand with the greater exponent. For example, if $x=4,320,000.$ and $y=12.1$ then $x+y$ is equal to:

$$\begin{split}
x & = 432 \times 10^{4} \\
y & = 000.00121 \times 10^{4} \\
x + y & = 432.00121 \times 10^{4} \quad \text{intermediate result} \\
      & = 432 \times 10^{4} \quad \text{final result}
\end{split}$$

In the above example, eight digits are required to compute the exact result. Using extra digits to compute the intermediate
sum or difference exactly is not practical in computer hardware; floating-point hardware normally operates using a fixed number
of digits.

Instead of using extra digits, suppose that we keep only the three digits that fit in the significand after scaling the operand with the smaller
exponent. The above calculation then becomes:

$$\begin{split}
x & = 432 \times 10^{4} \\
y & = 000 \times 10^{4} \quad \text{discard extra digits} \\
x + y & = 432 \times 10^{4}
\end{split}$$

which produces the same result compared to computing an exact intermediate sum or difference. But this is not always the case.
For example, if $x=10.5$ and $y=9.98$ then $z = x-y$ is equal to:

$$\begin{split}
x & = 105 \times 10^{-1} \\
y & = 099 \times 10^{-1} \quad \text{discard extra digits} \\
x - y & = 006 \times 10^{-1}
\end{split}$$

or $z = 600 \times 10^{-3}$ when converted to its normalized form. The true result is $\hat{z} = 520 \times 10^{-3}$ which is an
error of 80 ulps.

**Exercise 6** Compute the relative error in the above example.

**Exercise 7** The worst relative error when computing a difference of two different values occurs when the two values are almost
equal. What is the relative error when computing the value $x-y$ when $x=10.0$ and $y=9.99$ and the floating-point values
have three digits in the significand?

### Guard digits

As illustrated in the previous section, adding or subtracting two floating-point values can be inaccurate when the operands have
different orders of magnitude. Although it is impractical to add many extra digits to compute the intermediate result, it is possible
to add a small number of extra called *guard digits*. Even adding a single guard digit improves the error dramatically.
For example, if $x=10.5$ and $y=9.98$ then $z = x-y$ is equal to:

$$\begin{split}
x & = 105.0 \times 10^{-1} \quad \text{digit after decimal point is the guard digit} \\
y & = 099.8 \times 10^{-1} \quad \text{digit in smaller value preserved by guard digit} \\
x - y & = 005.2 \times 10^{-1} \\
      & = 520 \times 10^{-3} \quad \text{convert to normalized form and round}
\end{split}$$

which matches the true result of $\hat{z} = 520 \times 10^{-3}$.

Guard digits cannot eliminate rounding error. For example, if $x=110$ and $y=8.59$ then $z = x-y$ is equal to:

$$\begin{split}
x & = 110.0 \times 10^{0} \quad \text{digit after decimal point is the guard digit} \\
y & = 008.5 \times 10^{0} \quad \text{digit past the guard digit is discarded} \\
x - y & = 101.5 \times 10^{0} \\
      & = 102 \times 10^{0} \quad \text{convert to normalized form and round}
\end{split}$$

compared to the true value $\hat{z} = 101.41$.

**Exercise 8** Compute $0.9 + 100$ using a three digit significand and no guard digit. Repeat the calculation using a guard digit.

## Cancellation

When two similar values are subtracted, the resulting difference has a small magnitude. Two similar values will have identical digits
in their significands except in a few digits at the end of the significand (the least significant digits or the low
order digits). The digits at the beginning of the significand (the most significant digits or high order digits)
in the operands will cancel one another when the difference is computed. There are two kinds of cancellation: benign and catastrophic.

#### Benign cancellation

In the difference $x-y$, benign cancellation occurs when $x$ and $y$ do not contain rounding errors. In this case, the
result of the difference is as close to the true value as the floating-point representation allows.


#### Catastrophic cancellation

In the difference $x-y$, catastrophic cancellation occurs when $x$ and/or $y$ contain rounding errors; this occurs when $x$ and
$y$ are values computed using floating-point arithmetic. Students should complete the following exercises to see the effects
of catastrophic cancellation.

**Exercise 9** The quadratic formula involves computing the difference $b^2 - 4ac$. If $b^2$ and $4ac$ have similar values, then
catastrophic cancellation may occur because computing $b^2$ and $4ac$ involves floating-point operations that can result in
rounding errors. Compute the *exact* value of $b^2 - 4ac$ for $b=3.34$, $a=1.22$, and $c=2.28$.

**Exercise 10** Compute the floating-point values of $b^2$ and $4ac$ where the significand has three digits by rounding the exact values
so that they fit into the significand.

**Exercise 11** Using the result from the previous exercise, compute the difference $b^2 - 4ac$ using floating-point values.

**Exercise 12** Compute the relative error and error in ulps of the floating-point difference compared to the exact value.

In the above exercises, the difference $b^2 - 4ac$ is computed *without* rounding error, but the values $b^2$ and $4ac$ 
both contain rounding errors. Computing the difference does not cause the large error in the final result, but it does
expose the errors caused by computing $b^2$ and $4ac$ and rounding their values.

Another example of catastrophic cancellation is the function

$$f(x) = \frac{1 - \cos(x)}{x^2}$$

when $x \approx 0$. Even though $1$ contains no rounding error (and can be represented exactly as a floating-point value),
$\cos(x)$ must be computed and will contain rounding error. When $x \approx 0$ the value of the function is
$f(x) \approx 0.5$. Computing $f(10^{-8})$ using `double` arithmetic yields: 

In [1]:
double x = 1e-8;
double y = 1 - Math.cos(x);
double a = y / (x * x);
System.out.println(a);

0.0


null

which has a relative error of 1.

Sometimes, cancellation can be avoided by rewriting the expression so that it avoids subtraction. In the above example, the half angle formula

$$\sin^2 \left( \frac{\theta}{2} \right) = \frac{1 - \cos (\theta)}{2}$$

can be used to rewrite $f(x)$ as

$$f(x) = \frac{2\sin^2 \left( \frac{x}{2} \right) }{x^2}$$

which requires no subtraction. Computing $f(10^{-8})$ using `double` arithmetic and the rewritten formula yields:  

In [2]:
double x = 1e-8;
double y = 2 * Math.pow(Math.sin(x / 2), 2);
double a = y / (x * x);
System.out.println(a);

0.5


null

which has a relative error of approximately 0.

A third example of catastrophic cancellation is the function

$$f(x) = e^x - 1$$

where $x \approx 0$ and $e = 2.71828...$ is Euler's number. The `Math` class contains a method `expm1(double x)` specifically to calculate $f(x)$.
The next cell compares the results of `expm1(x)` and `exp(x) - 1` for a small value of `x`:

In [24]:
double x = 1e-12;
double a = Math.expm1(x);
double b = Math.exp(x) - 1;
System.out.println(a);
System.out.println(b);

1.0000000000005E-12
1.000088900582341E-12


null

Notice that almost all of the digits differ in the two results. The `expm1` and `log1p` methods are useful in certain financial calculations;
see [page 12 of WECSSKAFPA](https://docs.oracle.com/cd/E19957-01/800-7895/800-7895.pdf) for details.

<div class="alert alert-block alert-info">
    Some readers might wonder how <tt>expm1</tt> is implemented. See <a href="http://www.math.utah.edu/~beebe/reports/expm1.pdf">this document</a>
    for details.
</div>

**Exercise 13** The function $f(x) = \sqrt{1 + x} - 1$ exhibits catastrophic cancellation for small values of $x$. Evaluate $f(0.000001)$ using
`float` arithmetic. Note that `Math.sqrt` returns a `double` so you will need to cast the result back to `float`.

**Exercise 14** The function from Exercise 13 is mathematically equivalent to $g(x) = \frac{x}{\sqrt{1 + x} + 1}$ which involves no
subtraction. Evaluate $g(0.000001)$ using
`float` arithmetic. Note that `Math.sqrt` returns a `double` so you will need to cast the result back to `float`. The true value to
8 decimal places is $4.99999875 \times 10^{-7}$.

In [56]:
// Exercises 13 and 14


null

## Summing many values

Computing the sum of many floating-point values is required to solve many problems such as when computing statistics and in numerical integration.
Each addition can produce a rounding error of up to 0.5 ulps; thus, naively summing many terms can produce a final result that has a large
rounding error. For example, the following cell naively sums the elements of a list containing the value $1,000,000$ and one thousand
copies of the value $0.1$:

In [17]:
import java.util.ArrayList;
import java.util.List;

List<Float> t = new ArrayList<>();
t.add(1000000.0f);
for (int i = 0; i < 1000; i++) {
    t.add(Float.valueOf(0.1f));
}
float sum = 0.0f;
for (Float val : t) {
    sum += val;
}
System.out.println(sum);

1000100.0000014901


null

The correct sum is $1,000,100$ but the program outputs a different value. The main source of error in the sum is caused by the first addition
in the sum which adds $1,000,000$ to $0.1$. Because the two values differ by many orders of magnitude, the resulting rounding error is large compared
to the smaller term. You can see this by computing $1,000,000 + 0.1$ using `float` arithmetic and then casting the result to a `double`:

In [21]:
System.out.println((double) (1000000f + 0.1f));

1000000.125


null

Adding 999 copies of 0.1 into the sum compounds the error so that the final sum is quite inaccurate. Adding the $1,000,000$ to the sum as the
last operation minimizes the rounding error:

In [22]:
import java.util.ArrayList;
import java.util.List;

List<Float> t = new ArrayList<>();
for (int i = 0; i < 1000; i++) {
    t.add(Float.valueOf(0.1f));
}
t.add(1000000.0f);
float sum = 0.0f;
for (Float val : t) {
    sum += val;
}
System.out.println(sum);

1000100.0


null

Generalizing the above example, sorting the values to be summed from smallest to largest *in magnitude* is sometimes recommmended as
a method of minimizing rounding error when computing a sum of many values. Of course, sorting changes the order of the elements in 
the data array, which is often not desirable; in such cases, a copy of the original data source must be made which requires an
additional $O(n)$ memory. Sorting also has $O(n \log n)$ complexity whereas summing $n$ elements should ideally have $O(n)$ complexity.

A better way to compute the sum of many floating-point values is to use
[Kahan's summation algorithm](https://en.wikipedia.org/wiki/Kahan_summation_algorithm) shown below:

In [38]:
import java.util.List;

public class Sums {
    
    public static float kahanSum(List<Float> data) {
        float s = 0.0f;
        float c = 0.0f;
        for (float val : data) {
            float y = val - c;
            float t = s + y;
            c = (t - s) - y;
            s = t;
        }
        return s;
    }
}

com.twosigma.beaker.javash.bkr68cfd94c.Sums

Using Kahan's algorithm to sum the values in the previous example produces the expected value of `1000100.0`:

In [39]:
import java.util.ArrayList;
import java.util.List;

List<Float> t = new ArrayList<>();
t.add(1000000.0f);
for (int i = 0; i < 1000; i++) {
    t.add(Float.valueOf(0.1f));
}
System.out.println(Sums.kahanSum(t));

1000100.0


null

#### How Kahan's algorithm works

Kahan's algorithm keeps track of two running values:

* `s` : the accumulated sum
* `c` : the error caused by adding an element to the sum from the *previous* iteration

When `c` is positive, the current sum is too large because of rounding error, and when `c` is negative, the current sum
is too small because of rounding error. To compensate for the rounding error, `c` is subtracted from the next element that
is added to the sum on the line:

```java
float y = val - c;    // val is the next element to add to the sum
```

`t` is a temporary value equal to the current sum plus `y`:

```java
float t = s + y;      // add the next (error compensated) element to the sum
```

Because `t` involves an arithmetic operation of floating-point values, it contains rounding error.

The key in Kahan's algorithm is in estimating how much rounding error has accumulated by adding `y` to the sum, which occurs on the 
line:

```java
c = (t - s) - y;
```

Mathematically, `c` should be zero because:

$$\begin{split}
c & = (t - s) - y \\
  & = (s + y - s) - y \\
  & = y - y \\
  & = 0
\end{split}$$

However, the term `t` contains rounding error so that `(t - s)` is not exactly equal to `y`. In other words, mathematically:

$$c = y' - y$$

where $y' \approx (y + \text{rounding error caused by computing}\ s + y)$. Subtracting $y$ from $y'$ yields an approximate
estimate of the rounding error caused by adding the most recent element to the sum.
This rounding error is subtracted from the next element at the beginning of the next loop iteration before adding it the sum.

Kahan's algorithm is much more accurate than naive summation, but it cannot exactly compensate for all rounding errors.

<div class="alert alert-block alert-info">
    For interested readers, the <a href="https://en.wikipedia.org/wiki/Kahan_summation_algorithm">Wikipedia page</a>
    for Kahan's algorithm describes enhancements and alternatives to the algorithm. 
    <a href="https://www.cs.toronto.edu/~radford/ftp/xsum.pdf">More recent work</a> suggests that there are faster
    ways of computing accurate sums.
</div>

**Exercise 15** Consider a floating-point system that uses a three digit base-10 significand and one guard digit. Sum the values $100, 0.4, 0.4, 0.4, 0.4, 0.4$ using naive summation and Kahan's algorithm. Show the intermediate sum after adding each element to the sum and compare it to
the true answer.